In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

In [ ]:
df=pd.read_csv("Aggregate_covid_country.csv")
df

# EDA

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.describe()

Key Obseravtions here

mean and median have huge distance between them.
Also, 75% and max have huge gap . which shows outliers are present .
Now, different states may have different ratio of deaths due to covid. So , removing the outliers will not be good for data

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull())

There are no null values 


In [ ]:
#univariate Analysis

In [ ]:
sns.distplot(df["Confirmed"])
plt.title("Confirmed Cases")
plt.show()

In [ ]:
sns.distplot(df["Recovered"])
plt.title("Recovered Cases")
plt.show()

In [ ]:
sns.distplot(df["Deaths"])
plt.title("Deaths due to COVID")
plt.show()

In [ ]:
# univariate analysis shows all these features are positively skewed

In [ ]:
#now bivariate analysis using scatter plot to check the relationship between features and target variable- Deaths

In [ ]:
plt.scatter(df["Confirmed"],df["Deaths"])
plt.title("scatter plot")
plt.xlabel("Deaths")
plt.ylabel("Confirmed")
plt.show()

there is a positive linear relationship between confirmed cases and deaths due to covid

In [ ]:
plt.scatter(df["Recovered"],df["Deaths"])
plt.title("scatter plot")
plt.xlabel("Deaths")
plt.ylabel("Recovered")
plt.show()

there is a positive linear relationship between recovered cases and deaths due to covid

In [ ]:
#multivariate analysis

In [ ]:
corr=df.corr()
sns.heatmap(corr,annot=True)
plt.show()

#date variable may be dropped as it has no much related to predict deaths due to covid

In [ ]:
df.drop(["Date"],axis=1,inplace=True)
df

#transforming the categorical variables to numerical for data modelling using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
list1=['Country']
for val in list1:
    df[val]=le.fit_transform(df[val].astype(str))
    


In [ ]:
df

# handling Outliers

In [ ]:
sns.boxplot(data=df[["Confirmed","Recovered","Deaths"]],orient='h',palette='Set2')
plt.show()

In [ ]:
from scipy.stats import zscore
z=np.abs(zscore(df))
threshold=3
df_new=df[(z<3).all(axis=1)]

In [ ]:
df_new

In [ ]:
df.shape

In [ ]:
df_new.shape

In [ ]:
df_new.skew()

Skewness values are not in range (+/-)0.5. removing the skewness

In [ ]:
from sklearn.preprocessing import power_transform


In [ ]:
x=df_new.iloc[:,0:-1]
x.shape

In [ ]:
y=df_new.iloc[:,-1]
y.shape

In [ ]:
#x=power_transform(x,method='yeo-johnson')

In [ ]:
x

In [ ]:
#scaling the dataset
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
scaledX=sc.fit_transform(x)

In [ ]:
# finding the best random state

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

In [ ]:
best_rstate=0
accuracy=0
for i in range(30,200):
    x_train,x_test,y_train,y_test=train_test_split(scaledX,y,test_size=.25,random_state=i)
    mod=LinearRegression()
    mod.fit(x_train,y_train)
    predlr=mod.predict(x_test)
    tempaccur=r2_score(y_test,predlr)
    if tempaccur>accuracy:
        accuracy=tempaccur
        best_rstate=i
        
print(f"Best Accuracy {accuracy*100} found on random_state {best_rstate}")        

In [ ]:
# using the best random state

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(scaledX,y,test_size=.25,random_state=80)

 # Finding the best model

Linear Regression

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
predlr=lr.predict(x_test)
r2score=r2_score(y_test,predlr)
cvscore=cross_val_score(LinearRegression(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(x_train,y_train)
predrf=rf.predict(x_test)
r2score=r2_score(y_test,predrf)
cvscore=cross_val_score(RandomForestRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ab=AdaBoostRegressor()
ab.fit(x_train,y_train)
predab=rf.predict(x_test)
r2score=r2_score(y_test,predab)
cvscore=cross_val_score(AdaBoostRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor()
sgd.fit(x_train,y_train)
predsgd=rf.predict(x_test)
r2score=r2_score(y_test,predsgd)
cvscore=cross_val_score(SGDRegressor(),x_train,y_train,cv=5).mean()
print("Accuracy Score" ,{r2score*100} , "Cross Val Score", {cvscore*100})

# Conclusion
RandomForestRegressor is best performing model with r2Score 97%

In [ ]:
a=np.array(y_test)
a

In [ ]:
df_con=pd.DataFrame({'original':a,"predicted":predrf},index=range(len(a)))
df_con

# Saving the model- Serialization

In [ ]:
# saving the prediction model

import pickle
filename="CovidData.pkl"
pickle.dump(predrf,open(filename,'wb'))